In [1]:
import pandas as pd

In [2]:
desc_df = pd.read_excel("../data/desc.xlsx")

In [3]:
desc_df.head()

,project_id,project_address,desc_total,video_desc,risk_desc
0,804380059,/projects/1552697291/ufactory-3d-printing-shop/,Follow us on Facebook and Twitter 횂혻and pl...,my name is Matt Elkins not here to tell you a...,Risks and challenges The biggest challenge of...
1,1560420632,/projects/941562770/nervous-dating-dating-for-...,"""I get very nervous when I think someone is f...",introverts the idea of going on the site can ...,Risks and challenges Fulfilment of your 'free...
2,278626358,/projects/703843051/messenger-rock/,"Hi, do you feel an inner hatred that you wish...",NaN,Risks and challenges The rock quality. Learn ...
3,907761690,/projects/1407693875/wind-turbine-phone-charge...,The goal of my project is to build a wind tur...,NaN,Risks and challenges The risks and challenges...
4,1512099429,/projects/358359447/crash-surveillance-drone-csd/,The project will consist of building a protot...,NaN,Risks and challenges The initial project is t...


# Risk Perception

In [4]:
desc_df.head()

,project_id,project_address,desc_total,video_desc,risk_desc
0,804380059,/projects/1552697291/ufactory-3d-printing-shop/,Follow us on Facebook and Twitter 횂혻and pl...,my name is Matt Elkins not here to tell you a...,Risks and challenges The biggest challenge of...
1,1560420632,/projects/941562770/nervous-dating-dating-for-...,"""I get very nervous when I think someone is f...",introverts the idea of going on the site can ...,Risks and challenges Fulfilment of your 'free...
2,278626358,/projects/703843051/messenger-rock/,"Hi, do you feel an inner hatred that you wish...",NaN,Risks and challenges The rock quality. Learn ...
3,907761690,/projects/1407693875/wind-turbine-phone-charge...,The goal of my project is to build a wind tur...,NaN,Risks and challenges The risks and challenges...
4,1512099429,/projects/358359447/crash-surveillance-drone-csd/,The project will consist of building a protot...,NaN,Risks and challenges The initial project is t...


In [5]:
risk_desc_df = desc_df[['project_id', 'project_address']]

## Request

In [6]:
from selenium import webdriver

browser = webdriver.Chrome('chromedriver')

In [ ]:
from bs4 import BeautifulSoup

def parse_risk_text(fname):
    try:
        text = open(fname, "r").read()
        text = text.partition("Risks and challenges")[2]
        text = text.partition("Questions about this project?")[0]

        soup = BeautifulSoup(text, 'html.parser')
        matched = soup.findAll(attrs={'class':'js-risks-text'})[0].contents[0]

        return matched
    except:
        return ""

In [ ]:
import tqdm
import os
import codecs


cwd = os.getcwd()

risk_desc_list = []

for i, r in tqdm.tqdm(risk_desc_df.iterrows(), total=risk_desc_df.shape[0]):
    fpath = cwd + "/html/{}.html".format(i)
    try:
        if os.path.exists(fpath)==True:
            matched_text = parse_risk_text(fpath)

            if len(matched_text) ==0:
                url = 'https://www.kickstarter.com'+r['project_address']
                browser.get(url)  
                browser.set_page_load_timeout(45)
                browser.implicitly_wait(15)

                file_object = codecs.open(fpath, "w", "utf-8")
                html = browser.execute_script("return document.body.innerHTML;")                
                flag = file_object.write(html)

                matched_text = parse_risk_text(fpath)

        else:
            url = 'https://www.kickstarter.com'+r['project_address']
            browser.get(url)  
            browser.set_page_load_timeout(45)
            browser.implicitly_wait(10)

            file_object = codecs.open(fpath, "w", "utf-8")
            html = browser.execute_script("return document.body.innerHTML;")                
            flag = file_object.write(html)

            matched_text= browser.find_element_by_class_name('js-risks-text').text

        risk_desc_list.append(matched_text)

    except:
        print("Exception at {}".format(i))
        risk_desc_list.append('')    

 53%|█████▎    | 4041/7571 [46:07<1:16:53,  1.31s/it]

In [ ]:
browser.close()

In [ ]:
risk_desc_df.shape

In [ ]:
risk_desc_df['text'] = risk_desc_list

In [ ]:
risk_desc_df.to_pickle('risk_desc_df.pkl')

In [ ]:
risk_desc_df.to_csv('risk_desc_df.csv', encoding='utf-8')

In [ ]:
risk_desc_df.to_excel('risk_desc_df.xlsx', encoding='utf-8')